# huggingface datasets X Openprompt

In this notebook, you will learn how to use integrate huggingface datasets utilities into openprompt to enable prompt learning in more datasets.

In [17]:
from datasets import load_dataset
boolq_dataset = load_dataset('super_glue', 'boolq', cache_dir="../datasets/.cache/huggingface_datasets")


Using the latest cached version of the module from /home/hushengding/.cache/huggingface/modules/datasets_modules/datasets/super_glue/2fb163bca9085c1deb906aff20f00c242227ff704a4e8c9cfdfe820be3abfc83 (last modified on Tue Oct 19 07:57:21 2021) since it couldn't be found locally at super_glue, or remotely on the Hugging Face Hub.
Reusing dataset super_glue (../datasets/.cache/huggingface_datasets/super_glue___super_glue)/boolq/1.0.2/2fb163bca9085c1deb906aff20f00c242227ff704a4e8c9cfdfe820be3abfc83)


  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
boolq_dataset['test'][0]

{'label': -1,
 'passage': '20 euro note -- Until now there has been only one complete series of euro notes; however a new series, similar to the current one, is being released. The European Central Bank will, in due time, announce when banknotes from the first series lose legal tender status.',
 'idx': 0,
 'question': 'is the first series 20 euro note still legal tender'}

Not that You might need to specify a cache_dir for better data exploring

You might encounter "IProgress not found. Please update jupyter and ipywidgets."
Please run
`conda install -c conda-forge ipywidgets` and `jupyter nbextension enable --py widgetsnbextension`, then restart the jupyter notebook.

## Define (or load) a transformation

In [41]:
from openprompt.data_utils import InputExample

def transform(example):
    text_a = example['paragraph']
    text_b = example['question']
    meta = {"answer": example["answer"]}
    label = int(example['label'])
    guid = "p{}-q{}-a{}".format(example['idx']['paragraph'], example['idx']['question'], example['idx']['answer'])
    return InputExample(guid = guid, text_a = text_a, text_b = text_b, meta = meta, label=label)

## Or load a transformation

In [1]:
from openprompt.data_utils.huggingface_dataset import SuperglueMultircProcessor
train_dataset = SuperglueMultircProcessor().get_train_examples()


Reusing dataset super_glue (/home/hushengding/.cache/huggingface/datasets/super_glue/multirc/1.0.2/2fb163bca9085c1deb906aff20f00c242227ff704a4e8c9cfdfe820be3abfc83)


## Apply the transformation

In [2]:
# train_dataset = list(map(transform, multirc_dataset['train']))
# valid_dataset = list(map(transform, multirc_dataset['validation']))
# test_dataset = list(map(transform, multirc_dataset['test']))
train_dataset[0]

{
  "guid": "p0-q0-a0",
  "label": 0,
  "meta": {
    "answer": "Children, Gerd, or Dorian Popa"
  },
  "text_a": "While this process moved along, diplomacy continued its rounds. Direct pressure on the Taliban had proved unsuccessful. As one NSC staff note put it, \"Under the Taliban, Afghanistan is not so much a state sponsor of terrorism as it is a state sponsored by terrorists.\" In early 2000, the United States began a high-level effort to persuade Pakistan to use its influence over the Taliban. In January 2000, Assistant Secretary of State Karl Inderfurth and the State Department's counterterrorism coordinator, Michael Sheehan, met with General Musharraf in Islamabad, dangling before him the possibility of a presidential visit in March as a reward for Pakistani cooperation. Such a visit was coveted by Musharraf, partly as a sign of his government's legitimacy. He told the two envoys that he would meet with Mullah Omar and press him on  Bin Laden. They left, however, reporting to W

In [20]:
import torch
import torch.nn as  nn

class A(nn.Module):
    def __init__(self,):
        super().__init__()
        self.x = nn.Linear(3,4)
        self.y = nn.Linear(4,2)
    
    def parameters(self, recurse: bool = True):
        for name, param in self.named_parameters(recurse=recurse):
            if "x" not in name:
                yield param
    
    def forward(self, x):
        return self.y(self.x(x)).sum()
    # def register_parameter(self, name: str, param) -> None:
    #     print("here {}".format(name))
    #     if "x" in name:
    #         pass
    #     else:
    #         return super().register_parameter(name, param)

a = A()
out = a(torch.randn(2,3))
out.backward()

print([i for i in a.parameters()])
print(a.x.weight.grad, a.y.weight.grad)


[Parameter containing:
tensor([[ 0.3003,  0.2248, -0.3258, -0.2067],
        [-0.4044,  0.2044, -0.3495,  0.4446]], requires_grad=True), Parameter containing:
tensor([-0.2840, -0.2530], requires_grad=True)]
tensor([[ 0.0611,  0.2935, -0.1864],
        [-0.2517, -1.2096,  0.7681],
        [ 0.3961,  1.9033, -1.2087],
        [-0.1395, -0.6706,  0.4259]]) tensor([[-0.2350,  0.4802, -2.6798,  2.0192],
        [-0.2350,  0.4802, -2.6798,  2.0192]])
